# Libraries

In [1]:
import torch
from torch import nn

import torchvision
import torchvision.transforms as transforms

import cv2
from PIL import Image

/home/lucas/miniconda3/envs/image-denoising/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/lucas/miniconda3/envs/image-denoising/lib/python3.11/site-packages/torchvision/image.so: undefined symbol: _ZN3c106detail23torchInternalAssertFailEPKcS2_jS2_RKSs'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


# 1. Load ConvNext

In [49]:
convnext = torchvision.models.convnext.convnext_base(weights="IMAGENET1K_V1")

# Adjust to remove Linear layer
#convnext.classifier = torch.nn.Sequential(*(list(convnext.classifier.children())[:-1]))

# Remove Classifier block
convnext = torch.nn.Sequential(
    *(list(convnext.children())[:-1]),
    torch.nn.Flatten()
)

In [50]:
print(convnext)

Sequential(
  (0): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 128, kernel_size=(4, 4), stride=(4, 4))
      (1): LayerNorm2d((128,), eps=1e-06, elementwise_affine=True)
    )
    (1): Sequential(
      (0): CNBlock(
        (block): Sequential(
          (0): Conv2d(128, 128, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=128)
          (1): Permute()
          (2): LayerNorm((128,), eps=1e-06, elementwise_affine=True)
          (3): Linear(in_features=128, out_features=512, bias=True)
          (4): GELU(approximate='none')
          (5): Linear(in_features=512, out_features=128, bias=True)
          (6): Permute()
        )
        (stochastic_depth): StochasticDepth(p=0.0, mode=row)
      )
      (1): CNBlock(
        (block): Sequential(
          (0): Conv2d(128, 128, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=128)
          (1): Permute()
          (2): LayerNorm((128,), eps=1e-06, elementwise_affine=True)
          (3): Linear(in_fe

# 2. Build Transformer Encoder

In [4]:
encoder_layer = nn.TransformerEncoderLayer(d_model=512, nhead=8)
transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=6)

/home/lucas/miniconda3/envs/image-denoising/lib/python3.11/site-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


# 3. Read Image

In [29]:
img = Image.open("./sample.jpg")

In [30]:
transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                             std=[0.229, 0.224, 0.225])
])

In [33]:
preprocessed_image = transform(img)

In [34]:
preprocessed_image.shape

torch.Size([3, 224, 224])

## 3.1. Pass image to model

In [51]:
output = convnext(preprocessed_image.unsqueeze(0))

In [62]:
output.reshape(2,-1)

tensor([[ 0.3821, -0.2521,  0.1586,  ...,  0.1863, -0.0245, -0.1182],
        [ 0.1550,  0.1666, -0.1017,  ..., -0.1183, -0.1245,  0.1566]],
       grad_fn=<ViewBackward0>)